## Working with LIDAR datasets in `adaptivefiltering`

This notebook will explain how Lidar datasets are treated in `adaptivefiltering` by showcasing the most common use cases. If you are not yet familiar with Jupyter, check the [Introduction to Python+Jupyter notebook](python.ipynb) first.

The first thing to do in a Jupyter notebook that uses `adaptivefiltering` is to import the library:

In [ ]:
import adaptivefiltering as af

### Loading datasets

`adaptivefiltering` handles Lidar data sets in LAS/LAZ format. To load a data set, we construct a `DataSet` object given its filename and assign it to a variable `ds`:

In [ ]:
ds = af.DataSet(filename="500k_NZ20_Westport.laz")

In above example, we are loading a small sample data set that is provided by `adaptivefiltering`. You can also load your own data set by providing its filename. `adaptivefiltering` currently only supports datasets in LAS and LAZ format. The dataset filename is assumed to either be an absolute path, be located in the current working directory or that you first specified its location using the `set_data_directory` function:

In [ ]:
af.set_data_directory("some/directory", create_dir=True)

Here, the `create_dir` directory specifies whether `adaptivefiltering` should create non-existing directories for you.

### Spatial Reference Systems

By default, `adaptivefiltering` will try to determine the dataset's metadata to determine the correct spatial reference system. If it is not specified in the metadata or if you want to force interpretation as a certain spatial reference system, you can pass its [Well-known Text (WKT) representation](https://en.wikipedia.org/wiki/Well-known_text_representation_of_coordinate_reference_systems) or [EPSG code](https://epsg.io/) to the data set:

In [ ]:
ds_epsg = af.DataSet(filename="500k_NZ20_Westport.laz", spatial_reference="EPSG:4326")

Note that specifying a specific spatial reference system does *not* reproject the dataset, but reinterprets the given data. If you want to reproject your data, have a look at `adaptivefiltering.reproject_dataset` below.

### Visualizing datasets

With the dataset loaded as the object `ds`, we have several ways of visualizing the data set directly in Jupyter. By default, a hillshade model with a configurable spatial resolution in meters is used:

`adaptivefiltering` supports three different visualization methods, namly `Hillshade Model`, `Slope Map` and `Hillshade Model + Slope Map`. These can best be explored using an interactive user interface:

In [ ]:
ds.show_interactive()

If you already know exactly what visualization type and paramters you want, you can pass them directly to `show`. 

In [ ]:
ds.show(visualization_type="hillshade", resolution=0.5, classification="ground")

The full list of options is available in the [online documentation](https://adaptivefiltering.readthedocs.io/en/latest/index.html#adaptivefiltering.DataSet.show) or can be accessed directly in Jupyter by using the `?` operator:

In [ ]:
?ds.show

### Restricting datasets

If your Lidar dataset is very large, handling the entire data set becomes unwieldy, especially if we want to interactively tune ground point filtering pipelines. It is therefore important to crop the dataset to a subset that we can easily work on. We do so by showing an interactive map, adding a polygon with the polygon selector tool and hitting the *Finalize* button:

In [ ]:
rds = ds.restrict()

In the above, the restricted dataset is assigned to a new object `rds`. This follows a design principle of `adaptivefiltering`: All objects (datasets, filter pipelines etc.) are *immutable* - operations that work on datasets *never* implicitly modify an object. Instead the, provided input (`ds` in the above) is left untouched, and a modified copy is returned. This results in an increased memory consumption, but makes the interactive exploration of ground point filtering with `adaptivefiltering` easier to handle.

It is also possible to load a segmentation as a geojson file and overlay it ontop of the satellite image.


In [ ]:
segmentation_overlay = af.load_segmentation(
    "westportSegmentation.geojson", spatial_reference="EPSG:4326"
)

In [ ]:
rds = ds.restrict(segmentation_overlay=segmentation_overlay)

### Transforming datasets

The above principle of *immutability* is also followed by all other functions that transform datasets. The most prominent such transformation is the application of ground point filter pipelines. It is of such importance, that it is covered in in detail in [Selecting a filter pipeline](selection.ipynb) and [Creating filter pipelines](filtering.ipynb). Other data transformations are e.g. `remove_classification` which removes any existing classification data from a dataset:

In [ ]:
ds = af.remove_classification(ds)

Here, we have chosen to assign the transformed dataset to the same name as the original dataset. This is not violating the principle of immutability, because we explicitly chose to do so.

Another dataset transformation that was already mentioned is the reprojection into a different spatial reference system:

In [ ]:
reprojected = af.reproject_dataset(ds, "EPSG:4326")

If your dataset's metadata does not specify a spatial reference system, you need specify it additionally using the `in_srs=` parameter to `adaptivefiltering.reproject_dataset`.

### Saving datasets

Once we have achieved a result that is worth storing, we can save the dataset to a LAS/LAZ file by calling its `save` method:

In [ ]:
saved = ds.save("without_classification.las", overwrite=False)

In the above, the first argument is the filename to save to (relative paths are interpreted w.r.t. the current working directory). Optionally, LAZ compression can be activated by setting `compress=True`. If an existing file would be overwritten, explicit permission needs to do that needs to be granted by setting `overwrite=True`. The return object `saved` is again an `adaptivefiltering` dataset object that represents the LAS/LAZ file on the disk.